## *PhotoDissociation Region Toolbox Notebooks*
-------------------------------------------------------------

# Example 1: Working With Measurements

This Example shows how to bring your spectral line and far-infrarend (FIR) data into the PDR Toolbox.

First import the relevant modules.  

In [1]:
from pdrtpy.measurement import Measurement
from pdrtpy.modelset import ModelSet
import pdrtpy.pdrutils as utils
from astropy.nddata import StdDevUncertainty
import astropy.units as u
import numpy as np

/home/mpound/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:984: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(super(), op)(other)


## Measurements
To use PDR Toolbox, you need to create `Measurements` from your observations. A Measurement consists of a value and an error.  These can be single-valued or an array of values.  In the typical case of an image, the Measurement is a representation of a FITS file with two HDUs, the first HDU is the spatial map of intensity and the 2nd HDU is the spatial map of the errors.  It is based on [astropy's CCDData](https://docs.astropy.org/en/stable/api/astropy.nddata.CCDData.html) if you are familiar with that. Typical sub-millimeter maps we get from telescopes don't have the error plane, but PDRT makes it easy for you to create one if you know the magnitude of the error. Your FITS images can be in intensity units (equivalent to erg s$^{-1}$ cm$^{-2}$ sr$^{-1}$) or can be in K km/s.  PDRT will do appropriate conversion as necessary when it uses your images (original Measurement remains unchanged).

### Measurement Identifiers 
When you create a Measurement you have to say what it is a measurement of, i.e., what spectral line it is.  This is done using the string identifier.  The identifier (ID) should be one of the lines supported by the PDR Toolbox`ModelSets`.  The default ModelSet in the PDR Toolbox is the Wolfire/Kaufman 2006 constant extinction models.

In [2]:
ModelSet.WolfireKaufman().supported_lines

canonical name,ID
str51,str14
[C II] 158 $\mu$m,CII_158
[C I] 370 $\mu$m,CI_370
[C I] 609 $\mu$m,CI_609
CO(J=1-0),CO_10
CO(J=10-9),CO_109
CO(J=11-10),CO_1110
CO(J=12-11),CO_1211
CO(J=13-12),CO_1312
CO(J=14-13),CO_1413


## Creating a Measurement based on a single value

Suppose you have single-beam observations of [OI] 145 $\mu$m, [CI] 609 $\mu$m, CO(J=2-1), and [CII] 158 $\mu$m lines. You create Measurements for these using the constructor giving it the value, error, line identifier string, and units.   You can add optional beam size (bmaj,bmin,bpa), but note PDRT requires all Measurements have the same beam size before it can perform calculations.  

In [3]:
myunit = "erg s-1 cm-2 sr-1" # this is the default
m1 = Measurement(data=30.,uncertainty = StdDevUncertainty(5.0),identifier="OI_145",unit=myunit)
m2 = Measurement(data=10.,uncertainty = StdDevUncertainty(2.0),identifier="CI_609",unit=myunit)
m3 = Measurement(data=10.,uncertainty = StdDevUncertainty(1.5),identifier="CO_21",unit=myunit)
m4 = Measurement(data=100.,uncertainty = StdDevUncertainty(10.),identifier="CII_158",unit=myunit,bmaj=60*u.arcsec,bmin=60*u.arcsec,bpa=0*u.degree)
print(m1)
# You can convert any Measurement to equivalent units 
w = utils.to("W m^-2 sr^-1",m1)
print(w)
print("bmaj (deg): %.3e"%m4.header["BMAJ"])

30.0 +/- 5.0 erg / (cm2 s sr)
converting erg / (cm2 s sr) to W m^-2 sr^-1
0.03 +/- 0.005 W / (m2 sr)
bmaj (deg): 1.667e-02


## Creating Measurements from a FITS images

Typical sub-millimeter maps we get from telescopes don't have the error plane, but PDRT makes it easy for you to create one if you know the magnitude of the error or if you have a separate error map. Your FITS images can be in intensity units (equivalent to erg s$^{-1}$ cm$^{-2}$ sr$^{-1}$) or can be in K km s$^{-1}$.  PDRT will do appropriate conversion as necessary when it reads in your images.   Here is an example using the N22 data from [Jameson et al. 2018](https://ui.adsabs.harvard.edu/abs/2018ApJ...853..111J/abstract). These data consist of separate flux and error maps for [OI] 145 $\mu$m and [CII] 158 $\mu$m,  and a flux map of the total far-infrared emission (FIR) which has a 10% error.  The static method `Measurement.make_measurement` will put the flux and error together in a single image with 2 HDUs and write that FITS file to disk.  You then read the image back in via the `read` method.

In [4]:
# get the data from the testdata directory
cii_flux = utils.get_data("n22_cii_flux.fits")  # [C II] flux
cii_err = utils.get_data("n22_cii_error.fits")  # [C II] error
oi_flux = utils.get_data("n22_oi_flux.fits")    # [O I] flux 
oi_err = utils.get_data("n22_oi_error.fits")    # [O I] error
FIR_flux = utils.get_data("n22_FIR.fits")       # FIR flux

# Output file names
cii_combined = "n22_cii_flux_error.fits"
oi_combined = "n22_oi_flux_error.fits"
FIR_combined = "n22_FIR_flux_error.fits"

# create the Measurements and write out the FITS files.
# Set overwrite=True to allow multiple runs of this notebook.
Measurement.make_measurement(cii_flux, cii_err, cii_combined,overwrite=True)
Measurement.make_measurement(oi_flux, oi_err, oi_combined,overwrite=True)
# Assign a 10% error in FIR flux
Measurement.make_measurement(FIR_flux, error='10%', outfile=FIR_combined,overwrite=True)

# Read in the FITS files to Measurements
cii_meas = Measurement.read(cii_combined, identifier="CII_158")
FIR_meas = Measurement.read(FIR_combined, identifier="FIR")
oi_meas = Measurement.read(oi_combined, identifier="OI_63")

print(oi_meas.wcs)
print("[OI] min %.2E max %.2E  %s"%(np.nanmax(oi_meas.data), np.nanmin(oi_meas.data),oi_meas.unit))

WCS Transformation

This transformation has 2 pixel and 2 world dimensions

Array shape (Numpy order): (139, 81)

Pixel Dim  Data size  Bounds
        0         81  None
        1        139  None

World Dim  Physical Type  Units
        0  pos.eq.ra      deg
        1  pos.eq.dec     deg

Correlation between pixel and world axes:

           Pixel Dim
World Dim    0    1
        0  yes  yes
        1  yes  yes
[OI] min 2.48E-07 max 3.33E-09  W / (m2 sr)


## The more you have in the FITS header the less you have to specify

Here we read in the CO(J=1-0) integrated intensity map that has the units, error, and beam parameters specified by the FITS keywords BUNIT, RMS, BMAJ, BMIN, BPA respectively.  These are old Bell Labs 7-m data of the Draco High Latitude Cloud (MBM 41-44).


In [5]:
outfile="draco_CO_measurement.fits"
infile=utils.get_data("draco.ico.fits")
Measurement.make_measurement(infile,error='rms',outfile=outfile)
ico = Measurement.read(outfile,identifier="CO_10")
print(ico.unit)
ico.header

Found RMS in header: 3.00E-01 (1, 163, 138)
K km / s


SIMPLE  =                    T /                                                
BITPIX  =                  -32                                                  
NAXIS   =                    3 /                                                
NAXIS1  =                  138 /                                                
NAXIS2  =                  163 /                                                
NAXIS3  =                    1 /                                                
EXTEND  =                    T                                                  
CROTA1  =           0.00000000 /                                                
CROTA2  =           0.00000000 /                                                
CROTA3  =           0.00000000 /                                                
EPOCH   =           1950.00000 /                                                
BUNIT   = 'K km/s  '           /                                                
RMS     =           0.300000